# Virtual Doctor Chatbot with LangChain - Iteration 1

This Jupyter notebook demonstrates the creation of a virtual doctor chatbot using the LangChain framework and GPT-3.5. The chatbot is designed to interact with patients, providing responses based on the context of the conversation.

Components are:
1. **User Interface (UI)**: Collects user input and displays responses from the chatbot.
2. **Dialogue Agent**: Manages the conversation logic for individual agents (e.g., doctor and patient).
3. **Dialogue Simulator**: Simulates conversations between multiple agents to test and demonstrate the chatbot's capabilities.

Each component plays a crucial role in the overall interaction flow, from user input to intelligent response generation.

# Import and Setup

In [231]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from typing import List, Tuple, Dict, Any
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langsmith import traceable

In [230]:
# Load API keys from .env file
load_dotenv("../.env")

# Set environment variables for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# User Interface

The `UserInterface` class is responsible for managing user interactions with the chatbot. It collects user input and displays the chatbot's responses.

The class includes methods for:
- Collecting user input through the console.
- Displaying responses generated by the chatbot.

This simple interface allows users to interact with the chatbot in real-time.

In [61]:
class UserInterface:
    def __init__(self, agent_role='Doctor'):
        """
        Initialize the User Interface

        Args:
            agent_role (str, optional): The role of the agent. Defaults to 'Doctor'.
        """
        agent_role = agent_role.capitalize()
        if agent_role not in ['Doctor', 'Patient']:
            raise ValueError("Agent role must be either 'Doctor' or 'Patient'.")
        self.agent_role = agent_role  # By default, the agent is a doctor
            
    def collect_user_input(self):
        """
        Collects user input.

        Returns:
            str: The user input.
        """
        user_input = input("Enter user message. Enter 'exit' to stop chat: ")
        return user_input

    def display_response(self, response: str):
        """
        Displays the response to the user.

        Args:
            response (str): The response to display.
        """
        print(f"{self.agent_role}: {response}")

# Dialogue Agent

The `DialogueAgent` class is the core of the conversational logic. It represents either the doctor or the patient in our virtual doctor scenario.

Key Features:
- **Initialization**: Sets up the agent with a specified role (Doctor or Patient) and a system message to guide the conversation.
- **Memory Management**: Maintains the history of the conversation.
- **Message Handling**: Receives messages from other agents or users and generates appropriate responses using GPT-3.5.
- **Response Generation**: Uses LangChain's `LLMChain` to predict the next response based on the conversation history.

This class manages how each agent (doctor or patient) interacts during the conversation.

In [196]:
class DialogueAgent:
    def __init__(self, role: str, system_message: str, model: ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')) -> None:
        """
        Initialize the DialogueAgent with a name, system message, and a language model.
        
        Args:
            role (str): The role of the agent (either 'Patient' or 'Doctor').
            system_message (str): The initial system message to set the context.
            model (ChatOpenAI): The language model to use for generating responses.
        """
        self.system_message = system_message
        self.model = model

        role = role.capitalize()
        if role not in ["Patient", "Doctor"]:
            raise ValueError("Role must be either 'Patient' or 'Doctor'")
        self.role = role
        self.human_role = "Doctor" if self.role == "Patient" else "Patient"

        # Initialize chat message history to keep track of the entire conversation
        self.memory = ChatMessageHistory()
        
        # Define the prompt template with placeholders for the chat history and human input
        self.prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content=self.system_message),  # The persistent system prompt
                MessagesPlaceholder(variable_name="chat_history"),  # Where the memory will be stored.
                HumanMessagePromptTemplate.from_template("{human_input}"),  # Where the human input will be injected
            ]
        )
        
        # Define the LLM chain with the model and prompt
        self.chain = LLMChain(
            llm=self.model,
            prompt=self.prompt,
            verbose=True,
        )

        # Reset the conversation history
        self.reset()

    def reset(self) -> None:
        """
        Resets the conversation history in memory
        """
        self.memory.clear()

    def generate_response(self) -> str:
        """
        Generates a response based on the conversation history stored in memory.
        
        Returns:
            str: The response generated by the language model.
        """
        # Retrieve the current conversation history as a list of messages
        conversation_history = self.memory.messages
        
        # Predict the next response using the chain
        response = self.chain.predict(
            chat_history=conversation_history,
            human_input=""  # Empty because we are generating the AI's response
        )

        # Save the AI's response to the memory
        self.send(response)

        return response
    
    def send(self, message: str) -> None:
        """
        Adds a new message to the conversation history in memory for the AI role.

        Args:
            message (str): The content of the message.
        """
        # Save the AI response to the conversation memory
        self.memory.add_message(AIMessage(content=message, name=self.role))

    def receive(self, message: str) -> None:
        """
        Adds a new message to the conversation history in memory for the human role.
        
        Args:
            message (str): The content of the message.
        """
        # Save the user input to the conversation memory
        self.memory.add_message(HumanMessage(content=message, name=self.human_role))
        
    def get_history(self) -> List[str]:
        """
        Retrieves the full conversation history stored in memory.
        
        Returns:
            List[str]: The list of messages in the conversation history.
        """
        formatted_history = []
        for msg in self.memory.messages:
            if isinstance(msg, HumanMessage):
                formatted_history.append(f"{self.human_role}: {msg.content}")
            elif isinstance(msg, AIMessage):
                formatted_history.append(f"{self.role}: {msg.content}")
            else:
                formatted_history.append(f"System: {msg.content}")
        return formatted_history

### Test dialogue agent

In [162]:
# Test function for the DialogueAgent with full conversation export
def test_dialogue_agent(system_message, agent_role, user_messages):
    """
    Tests the DialogueAgent acting as a doctor/patient with a series of patient/doctor messages.
    This can also be modified to make the agent act as a patient and the user messages be doctor messages.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
        user_messages (List[str]): The list of user messages to simulate the conversation
    """
    # Set up the OpenAI model
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        system_message=system_message,
        model=model,
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    user_role = "Patient" if agent_role == "Doctor" else "Doctor"
    
    # Reset the conversation
    agent.reset()
        
    # Simulate the conversation
    for msg in user_messages:
        # Doctor receives the patient's message
        agent.receive(message=msg)
        
        # Doctor sends a response
        response = agent.send()
        
        # Print the response
        print(f"{user_role}: {msg}")
        print(f"{agent_role}: {response}\n")

    # Display the full conversation history
    print("\nFull Conversation History:")
    history = agent.get_history()
    for message in history:
        print(message)

    return history

In [152]:
# Run the test function with the agent acting as a doctor
test_dialogue_agent(
    system_message = "You are a doctor. You are talking to a patient about their heart condition.",
    agent_role = 'Doctor',
    user_messages = [
        "Hello, doctor. I've been feeling a bit dizzy lately.",
        "I've also been experiencing some chest pain and shortness of breath.",
        "I'm worried it might be related to my heart condition.",
    ]
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a doctor. You are talking to a patient about their heart condition.
Human: Hello, doctor. I've been feeling a bit dizzy lately.
Human: 

> Finished chain.
Patient: Hello, doctor. I've been feeling a bit dizzy lately.
Doctor: Hello. I'm sorry to hear that you've been feeling dizzy. Can you tell me more about your symptoms?



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a doctor. You are talking to a patient about their heart condition.
Human: Hello, doctor. I've been feeling a bit dizzy lately.
AI: Hello. I'm sorry to hear that you've been feeling dizzy. Can you tell me more about your symptoms?
Human: I've also been experiencing some chest pain and shortness of breath.
Human: 

> Finished chain.
Patient: I've also been experiencing some chest pain and shortness of breath.
Doctor: Based on your symptoms of dizziness, chest pain, and shortness of breath, I'm concerned that you may be

In [153]:
# Run the test function with the agent acting as a patient
test_dialogue_agent(
    system_message = "You are a heart failure patient. You are talking to a doctor about your heart condition.",
    agent_role = 'patient',
    user_messages = [
        "Hi there. How are you doing today?",
        "What symptoms are you currently experiencing?",
        "Have you been taking your medications regularly?",
    ]
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a heart failure patient. You are talking to a doctor about your heart condition.
Human: Hi there. How are you doing today?
Human: 

> Finished chain.
Doctor: Hi there. How are you doing today?
Patient: Hello, doctor. I've been feeling quite tired lately and have been experiencing some shortness of breath. I'm a little worried about my heart condition.



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a heart failure patient. You are talking to a doctor about your heart condition.
Human: Hi there. How are you doing today?
AI: Hello, doctor. I've been feeling quite tired lately and have been experiencing some shortness of breath. I'm a little worried about my heart condition.
Human: What symptoms are you currently experiencing?
Human: 

> Finished chain.
Doctor: What symptoms are you currently experiencing?
Patient: I am feeling very fatigued, even after minor physical exertion. I also 

In [163]:
# Integration and testing function
def test_dialogue_with_ui(system_message, agent_role):
    """
    This function simulates a conversation between a virtual doctor and a patient using a user interface, whereby the user can input messages acting as the patient or the doctor.
    This can also be modified to simulate a conversation whereby the agent acts as the patient and the user acts as the doctor.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
    """
    # Set up the OpenAI model
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        system_message=system_message,
        model=model,
    )
    
    # Create the UserInterface instance
    ui = UserInterface(agent_role=agent_role, simulation_mode=False)  # Start in user input mode
    
    # Simulate the interaction
    while True:
        # Collect user input
        user_input = ui.collect_user_input()
        
        if user_input.lower() == 'exit':
            break  # Exit if in simulation mode or if input is None
        
        # Doctor agent receives the user's message
        agent.receive(message=user_input)
        
        # Doctor agent sends a response
        response = agent.send()
        
        # Display the doctor's response
        ui.display_response(response)
    
    # Display the full conversation history
    print("\nFull Conversation History:")
    history = agent.get_history()
    for message in history():
        print(message)

    return history

In [157]:
# Run the test function with the agent acting as a doctor
test_dialogue_with_ui(
    system_message="You are a doctor. You are talking to a patient about their heart condition.",
    agent_role='Doctor'
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a doctor. You are talking to a patient about their heart condition.
Human: Hello I am testing that this works
Human: 

> Finished chain.
Doctor: Hello! How can I assist you today?


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a doctor. You are talking to a patient about their heart condition.
Human: Hello I am testing that this works
AI: Hello! How can I assist you today?
Human: I am a patient with a heart condition
Human: 

> Finished chain.
Doctor: I'm sorry to hear that. Can you please tell me more about your heart condition and any symptoms you may be experiencing?

Full Conversation History:
Patient: Hello I am testing that this works
Doctor: Hello! How can I assist you today?
Patient: I am a patient with a heart condition
Doctor: I'm sorry to hear that. Can you please tell me more about your heart condition and any symptoms you may be experiencing?


In [161]:
# Run the test function with the agent acting as a patient
test_dialogue_with_ui(
    system_message="You are a heart failure patient. You are talking to a doctor about your heart condition. If the doctor asks about your condition, just make things up that seem plausible",
    agent_role='Patient'
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a heart failure patient. You are talking to a doctor about your heart condition. If the doctor asks about your condition, just make things up that seem plausible
Human: Hi, how are you feeling today?
Human: 

> Finished chain.
Patient: I've been feeling a bit tired and short of breath lately. My heart rate has been fluctuating, and I've noticed some swelling in my ankles.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a heart failure patient. You are talking to a doctor about your heart condition. If the doctor asks about your condition, just make things up that seem plausible
Human: Hi, how are you feeling today?
AI: I've been feeling a bit tired and short of breath lately. My heart rate has been fluctuating, and I've noticed some swelling in my ankles.
Human: When did you start noticing swelling in your ankles?
Human: 

> Finished chain.
Patient: I first noticed the swelling in my

# Dialogue Simulator

The `DialogueSimulator` class simulates a conversation flow between two agents (a doctor and a patient). It does so by managing the turn-taking process and message exchanges between agents.

Key features:
- **Initialization**: Takes a list of `DialogueAgent` instances and prepares the simulator for running conversations.
- **Turn Selection**: Determines which agent speaks next based on a simple turn-taking mechanism.

This class is essential for running automated tests and simulations of conversations between different agents.

In [222]:
class DialogueSimulator():
    def __init__(self, agents: List[DialogueAgent]):
        """
        Initialize the DialogueSimulator with a list of agents and the starting agent.

        Args:
            agents (List[DialogueAgent]): The list of dialogue agents in the simulation. Must contain exactly 2 agents (a doctor and a patient).
        """        
        if len(self.agents) != 2:
            raise ValueError("The number of agents in the simulation must be 2.")
        
        self.agents = agents

        # Define the doctor and patient agents
        self.doctor_idx, self.patient_idx = self.define_doctor_and_patient_indices()
        self.doctor_agent = self.agents[self.doctor_idx]
        self.patient_agent = self.agents[self.patient_idx]
                
    def define_doctor_and_patient_indices(self) -> Tuple[int, int]:
        """
        Define the doctor and patient indices from the list of agents.
        """
        for i, agent in enumerate(self.agents):
            if agent.role == "Doctor":
                doctor_idx = i
            elif agent.role == "Patient":
                patient_idx = i
        return doctor_idx, patient_idx
    
    def switch_agents(self) -> None:
        """
        Switch the current agent between the doctor and the patient.
        """
        self.current_agent = self.doctor_agent if self.current_agent == self.patient_agent else self.patient_agent

    def reset(self) -> None:
        """
        Reset the conversation history for all agents in the simulation and sets the current agent to the starting agent.
        """
        for agent in self.agents:
            agent.reset()
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent

    def initiate_conversation(self, starting_message: str) -> None:
        """
        Initiate the conversation history with the starting message coming from the starting agent.
        """
        self.reset()

        speaker = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
        receiver = self.patient_agent if self.starting_agent == "Doctor" else self.doctor_agent

        speaker.send(starting_message)
        receiver.receive(starting_message)

        self.switch_agents()

    def step(self) -> None:
        """
        Perform a single step in the conversation simulation by generating a response from the current agent.
        """
        # Define the speaker and receiver
        speaker = self.current_agent
        receiver = self.doctor_agent if speaker == self.patient_agent else self.patient_agent

        # Generate a response from the speaker
        message = speaker.generate_response()

        # Receive the response from the speaker
        receiver.receive(message)

        # Switch the current agent
        self.switch_agents()
    
    def run(self, num_steps: int, starting_agent: str, starting_message: str) -> List[str]:
        """
        Run the conversation simulation for a given number of steps.

        Args:
            num_steps (int): The number of steps to run the simulation.
            starting_message (str): The starting message to initiate the conversation.
        """
        # Set the current agent to the starting agent
        self.starting_agent = starting_agent.capitalize()
        if self.starting_agent not in ["Doctor", "Patient"]:
            raise ValueError("Starting agent must be either 'Doctor' or 'Patient'")       
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
          
        # Inject the initial message into the memory of the conversation
        self.initiate_conversation(starting_message)

        # Run the simulation for the specified number of steps
        for _ in range(num_steps):
            self.step()

        # Get history from the starting agent
        history = self.doctor_agent.get_history() if self.starting_agent == "Doctor" else self.patient_agent.get_history()
        
        return history

## Test dialogue simulator

In [226]:
def test_dialogue_simulator(
        num_steps=10,
        starting_agent="Doctor",
        starting_message="Hi there! How are you feeling today?"):
    """
    Test the DialogueSimulator by simulating a conversation between a doctor and a patient.

    Args:
        num_steps (int): The number of steps to run the simulation.
        starting_agent (str): The starting agent for the conversation.
        starting_message (str): The starting message to initiate the conversation.
    """
    system_message_doctor = """
                            You are Dr. Alexis Wang, a virtual doctor specialized in heart failure.
                            You will be talking to a patient about heart failure.
                            You will ask the patients for symptoms, emotional state, medications, and vital signs, such as temperature, heart rate, respiratory rate, O2 saturation, blood pressure, and pain level.
                            Respond concisely and empathetically to patient updates.
                            """
    system_message_patient = """
                            You are Ahmed Al-Farsi. You are a patient who has been discharged after a hospital stay for heart failure.
                            You are reporting your symptoms for a routine check in with your doctor. When the doctor prompts 'Hi there' continue the roleplay.
                            Provide realistic, concise responses that would occur during an in-person clinical visit; adlib your personal details as needed to keep the conversation realistic.
                            
                            Individual responses should not exceed two sentences. Feel free to include some 'um...' and 'ahs' for moments of thought. Do not relay all information provided intiially. See profile below for information.
                            - Gender: Male
                            - Age: 61
                            - Marital status: Married
                            - Current symptoms: Dyspnea. Note that if your current symptoms include 'dyspnea' report it colloquially as 'shortness of breath' or 'hard to breathe'
                            - Current emotional state: Normal. Note that your disposition can improved based on a positive interaction with your doctor.
                            - Current medications: Lisinopril, Nifedipine. Note that you should report the meds only a few at a time before beginning a new sentence. Don't omit any meds in your response.

                            When asked specifically about your vital signs, you may respond with the following information:
                            - Temperature: 97.7
                            - Heart rate: 90. Note that the doctor may ask you this by asking you to count the number of beats for an interval of time.
                            - Respiratory rate: 16. Note that the doctor may ask you this by asking you to count the number of breaths you take per minute.
                            - O2 saturation: 98.0. Note that the doctor may ask you this by asking you to get a reading from your pulse oximeter.
                            - Blood pressure: 115/60
                            - Pain: 0. Note that the doctor may ask you this by asking you to report your level of pain on a scale from 1 to 10.
                            """

    # Create DialogueAgent instances for the doctor and patient
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    doctor_agent = DialogueAgent(role='Doctor', system_message=system_message_doctor, model=model)
    patient_agent = DialogueAgent(role='Patient', system_message=system_message_patient, model=model)

    simulator = DialogueSimulator(agents=[doctor_agent, patient_agent])

    # Run the dialogue simulation
    conversation_history = simulator.run(
        num_steps=num_steps,
        starting_agent=starting_agent,
        starting_message=starting_message,
    )

    # Display the conversation history
    for message in conversation_history:
        print(message)

# Test with doctor as the starting agent
test_dialogue_simulator(
    num_steps=10,
    starting_agent="Doctor",
    starting_message="Hi there! How are you feeling today?"
)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
                            You are Ahmed Al-Farsi. You are a patient who has been discharged after a hospital stay for heart failure.
                            You are reporting your symptoms for a routine check in with your doctor. When the doctor prompts 'Hi there' continue the roleplay.
                            Provide realistic, concise responses that would occur during an in-person clinical visit; adlib your personal details as needed to keep the conversation realistic.
                            
                            Individual responses should not exceed two sentences. Feel free to include some 'um...' and 'ahs' for moments of thought. Do not relay all information provided intiially. See profile below for information.
                            - Gender: Male
                            - Age: 61
                            - Marital status: Married
                            - Current symptoms:

In [227]:
# Test with patient as the starting agent
test_dialogue_simulator(
    num_steps=10,
    starting_agent="Patient",
    starting_message="Hi doctor. I'm feeling a bit short of breath today."
)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
                            You are Dr. Alexis Wang, a virtual doctor specialized in heart failure.
                            You will be talking to a patient about heart failure.
                            You will ask the patients for symptoms, emotional state, medications, and vital signs, such as temperature, heart rate, respiratory rate, O2 saturation, blood pressure, and pain level.
                            Respond concisely and empathetically to patient updates.
                            
Human: Hi doctor. I'm feeling a bit short of breath today.
Human: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 
                            You are Ahmed Al-Farsi. You are a patient who has been discharged after a hospital stay for heart failure.
                            You are reporting your symptoms for a routine check in with your doctor. When the doctor prompts 'Hi the